## Обкачивание страницы с помощью selenium

#### Подключаемся к странице и открываем оригинальный текст:

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get("%TEXT%")
# %TEXT% - ссылка на интересующий перевод с https://lyricstranslate.com

In [ ]:
org_lyr_button = driver.find_element_by_id('lyrics-preview')

In [ ]:
org_lyr_button.click()

#### Скачиваем текст:

In [ ]:
from collections import defaultdict
from pprint import pprint

In [ ]:
def downloading_lyrics(drvr):
    error = False
    i = 0 # параграф
    j = 1 # строка
    endcount = 0
    lyrics = defaultdict(list)

    while True:
        cur = 'll-' + str(i) + '-' + str(j)
        line = drvr.find_elements_by_class_name(cur)
        if len(line) == 0:
            if endcount == 0: # закончился абзац 
                i += 1
                j = 1
                endcount = 1
            elif endcount == 1: # закончилась песня
                endcount = 2
            elif endcount == 2:
                break
        elif len(line) == 1:
            error = True
            j += 1
            endcount = 0
        else:
            lyrics[cur.strip('ll-')] = [line[0].text, line[1].text]
            j += 1
            endcount = 0
            
    return lyrics, error

In [ ]:
lyrics = downloading_lyrics(driver)

#### Скачиваем метаданные:

Обратите внимание, что это не те метаданные, которые нужны для .json Цакорпуса (которые требуют разметки вручную). Однако эти метаданные позволяют (а) не перепутать тексты; (б) ускорить процесс разметки нужных метаданных.

In [ ]:
meta = {}

translation_node = driver.find_element_by_class_name('translate-node-text').text
song_node = driver.find_element_by_class_name('song-node-text').text

pattern_org = 'Добавлено .*? в ..\, [0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9] \- [0-9][0-9]:[0-9][0-9]'
pattern_tr = 'Добавлено .*? в ..\, [0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9] \- [0-9][0-9]:[0-9][0-9]'

org_meta = re.search(pattern_org, song_node)
tr_meta = re.search(pattern_tr, translation_node)

org_date = re.search('[0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9]', org_meta[0])[0]
tr_date = re.search('[0-9][0-9]\/[0-9][0-9]\/[0-9][0-9][0-9][0-9]', tr_meta[0])[0]
translator = re.search('Добавлено .*? в', tr_meta[0])[0][10:-2]

meta['original_song_date'] = org_date
meta['translation_date'] = tr_date
meta['translator'] = translator

# вот это бы надо доделать, там ещё куча ерунды бывает, надо брать либо до • либо до \n
meta['artist'] = driver.find_element_by_class_name(
    'song-node-info-artist').text.split(
    '•')[0].strip('Исполнитель: ')

meta['song'] = driver.find_element_by_class_name(
    'song-node-info-album').text.split(
    '•')[0].strip('Песня: ')

meta['album'] = driver.find_element_by_class_name(
    'song-node-info-album').text.split(
    '•')[1].strip('Альбом: ')

#### Загрузка текстов в файл

konlpy не пожелал устанавливаться на мой ноутбук (проблемы с Microsoft Visual C++, которые мы не смогли решить в своё время на программировании), а в коллабе не очень понятно, как подключить драйвер к браузеру, поэтому в этой программе я ещё и скачаю всё это в файл .json, который буду считывать уже в коллабе для морфологической обработки и создания XML.

Тут нужно ещё будет отдельно подумать, как передавать мета-информацию.

In [ ]:
import json

In [ ]:
file_name = meta['song'] + ' by ' + meta['artist']
with open(file_name + ".json", "w", encoding='utf-8') as write_file:
    json.dump([meta, lyrics[0]], write_file, ensure_ascii=False, indent=3)

Дальнейшая работа происходит в K-pop songs analizer part 2.